<a href="https://colab.research.google.com/github/PernasA/TP2_SOA/blob/master/HPC/Cuaderno_2_Martes_grupo6a_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### 1. Introducción

El Ejercicio 2 del TP3 tiene como objetivo probar la diferencia de velocidad de procesamiento que hay entre procesamiento secuencial con paralelo. Para ello, se generará un vector con valores enteros aleatorios y se pedirá al usuario que defina el tamaño del mismo. También se le solicitará que ingrese el número que se quiere buscar en el vector.
Con estos datos, el programa buscará al elemento y si lo encuentra dirá cuál fue la última posición donde apareció.

### 2. Armado del ambiente


In [ ]:
 #@markdown ### Seleccione el número a buscar:
 nro_a_buscar = 21 #@param {type:"slider", min:0, max:100, step:1}

#@markdown ---
#@markdown ### Seleccione el tamaño del vector:
tam_vector = 200000 #@param {type:"slider", min:1, max:200000, step:10}


### 3. Desarrollo
# CPU


In [ ]:
#--------------------------------------BIBLIOTECAS--------------------------------------#
import matplotlib.pyplot as plt
from datetime import datetime
import numpy
import sys
try:

  #@title Ingrese la url de una imagen:
  #Obtencion del tiempo inicial
  tiempo_total = datetime.now()
  
  #Declaracion de funcion que realiza el pasaje del tiempo obtenido mediante datetime.now(), a milisegundos.
  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

  #---------------------------------------CODIGO--------------------------------------------#
  # Definimos el vector y lo cargamos con valores aleatorios del 0 al 100

  vector_cpu= numpy.random.randint(0, 100, size = tam_vector)
  vector_cpu = vector_cpu.astype(numpy.int32())

  #print("El vector es: ",vector_cpu.tolist())

  tiempo_cpu = datetime.now()

  #Algoritmo aplicado en cpu
  ultima_aparicion = -1
  contador = 1
  for numero in vector_cpu:
    contador+=1
    if numero == nro_a_buscar:
      ultima_aparicion += contador



  print( "Cantidad de elementos:", len(vector_cpu) )
  print("Tiempo CPU:", tiempo_en_ms( datetime.now() - tiempo_cpu   ), "[ms]" )

  if ultima_aparicion != -1:
    print("Se encontró por CPU el valor:",nro_a_buscar)
    print("La última posición donde aparece es:", ultima_aparicion)
  else:
    print("No se encontró por CPU el valor:",nro_a_buscar)

  #Calculo del tiempo total de procesamiento de la imagen en CPU
  tiempo_cpu = datetime.now() - tiempo_cpu

  #Calculo del tiempo total de ejecucion del programa
  tiempo_total = datetime.now() - tiempo_total

  print("Tiempo total de ejecucion: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  print("Tiempo de procesamiento en CPU: ", tiempo_en_ms( tiempo_cpu   ), "[ms]" )

except Exception as e:
  print("Error", e )
  sys.exit()

Cantidad de elementos: 200000
Tiempo CPU: 732.59 [ms]
Se encontró por CPU el valor: 21
La última posición donde aparece es: 200708281
Tiempo total de ejecucion:  751.101 [ms]
Tiempo de procesamiento en CPU:  735.061 [ms]


# GPU

In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.7 MB 26.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 66 kB 4.6 MB/s 
     |████████████████████████████████| 75 kB 3.9 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=627362 sha256=3f1eb713cc4a49acebc70c1fa6fe077ea1ed0e1a4ff0fb6c8e019dc5067a4085
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2021.2.9-py2.py3-none-any.whl size=62370 sha256=c15de90029108cfb8d79ea804df88293ad2b8ad0dda13e6c4629cb55b69911ee
  Stored in directory: /root/.cache/pip/wheels/41/b9/6e/94bb014f6484b15ec77e7877f3a227609481ffd98db364504d
Successfully built pycuda pytools


In [ ]:
try:
  %matplotlib inline
  from datetime import datetime
  tiempo_total = datetime.now()

  import matplotlib.pyplot as plt
  import sys
  import numpy
  import pycuda.driver as cuda
  import pycuda.autoinit
  from pycuda.compiler import SourceModule

  tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0

  # Definimos el vector y lo cargamos con valores aleatorios del 0 al 100
  valor_aux = numpy.zeros((tam_vector,), dtype=int);
  valor_aux = valor_aux.astype(numpy.int32())

  vector_cpu= numpy.random.randint(0, 100, size = tam_vector)

  vector_cpu = vector_cpu.astype(numpy.int32())

  vec_ult_pos_cpu= numpy.empty_like(valor_aux)

  #Reservo memoria en los vectores de gpu
  vec_ult_aparicion_gpu = cuda.mem_alloc(vec_ult_pos_cpu.nbytes)
  vector_gpu               = cuda.mem_alloc( vector_cpu.nbytes )

  #Copio la memoria al gpu
  cuda.memcpy_htod( vec_ult_aparicion_gpu, valor_aux)
  cuda.memcpy_htod( vector_gpu, vector_cpu )

  # CPU - Defino la función kernel que ejecutará en GPU.
  module = SourceModule("""
  __global__ void kernel_axpy( int tam_vector, int *vector, int nro_a_buscar, int *cant_apariciones )
  {
    int idx = threadIdx.x + blockIdx.x*blockDim.x;
    if( idx < tam_vector )
    {
      if (vector[idx] == nro_a_buscar) {
        cant_apariciones[idx]+=1;
      }
    }
  }
  """)
  # CPU - Genero la función kernel.
  kernel = module.get_function("kernel_axpy")

  tiempo_GPU_img = datetime.now()

  # GPU - Ejecuta el kernel.
  # TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
  dim_hilo = 100
  dim_bloque = numpy.int( (len(vector_cpu)+dim_hilo-1) / dim_hilo )

  #print("el vector de cpu es: ",vector_cpu.tolist())

  #TODO: Ojo, con los tipos de las variables en el kernel.
  kernel( numpy.int32(len(vector_cpu)), vector_gpu, 
          numpy.int32(nro_a_buscar), 
          vec_ult_aparicion_gpu,
          block=( dim_hilo, 1, 1 ),
          grid= (dim_bloque, 1, 1) )

  cuda.memcpy_dtoh( vec_ult_pos_cpu, vec_ult_aparicion_gpu )

  result= vec_ult_pos_cpu.tolist()
  #print("el vector resultado es: ",result)


  print( "Cantidad de elementos:", len(vector_cpu) )
  print( "Thread x:", dim_hilo,", Bloque x:", dim_bloque )

  ult_posicion = -1
  contador_fin = 1
  for numero in result:
    contador_fin += 1
    if numero == 1:
      ult_posicion = contador_fin

  if ult_posicion != -1:
    print("Se encontró por GPU el valor:",nro_a_buscar)
    print("La última posición donde aparece es:",ult_posicion)
  else:
    print("No se encontró por GPU el valor:",nro_a_buscar)

  tiempo_GPU_img = datetime.now() - tiempo_GPU_img

  tiempo_total = datetime.now() - tiempo_total

  print("Tiempo TOTAL: ", tiempo_en_ms( tiempo_total ), "[ms]" )
  print("Tiempo GPU  : ", tiempo_en_ms( tiempo_GPU_img ), "[ms]" )
except Exception as e:
   print("Error: ", e)
   sys.exit()

Error:  cuInit failed: no CUDA-capable device is detected


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# 4. Métricas

Cant Elementos|Tiempo GPU | Tiempo CPU 
--------|-----------|------------------
10|2.934 [ms]      | 2.274 [ms]
1.000| 3.411[ms]     | 3.509 [ms]
10.000| 6.267 [ms]      |17.849 [ms]
100.000|30.502 [ms]       | 159.546 [ms]
200.000|53.803 [ms]       | 399.014 [ms]


Tiempo promedio GPU |	Tiempo promedio CPU
-------------------|------------------
16.15 ms | 97,2 ms


# 4. Conclusiones

Observando las metricas, podemos concluir que el procesamiento de la imagen en GPU se realiza más rapido que en CPU a medida que aumenta el tamaño del vector. Al inicio, incluso es más rápido el tiempo en cpu en un vector de tamaño 10. Sin embargo, ya cuando el vector es de 200.000 elementos, el tiempo en gpu es 4 veces más chico. Esto demuestra cómo el procesamiento paralelo es mucho mejor que el secuencial a medida que hay más datos para procesar. 
Otro punto a tener en cuenta fue la dificultad de implementar los algoritmos en gpu, mientras que en cpu es un simple loop.

# 5. Bibliografía

[1] Manejo de vectores -Numpy: https://numpy.org/doc/stable/reference/generated/numpy.zeros.html


[2] Cuda - Documentación Oficial Nvidia:  https://docs.nvidia.com/cuda/

[3] Python basico UNLaM: https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb
